# Circuit simulation examples (with synaptic manipulations)

In [ ]:
import obi_one as obi
from pathlib import Path

In [ ]:
circuit_path_prefix = Path("../../data/tiny_circuits")

### Loading a circuit

In [ ]:
circuit_name = "N_10__top_nodes_dim6"
circuit = obi.Circuit(name=circuit_name, path=str(circuit_path_prefix / circuit_name / "circuit_config.json"))
print(f"Circuit '{circuit}' with {circuit.sonata_circuit.nodes[circuit.default_population_name].size} neurons and {circuit.sonata_circuit.edges[circuit.default_edge_population_name].size} synapses")

### Set up a simulation campaign

In [ ]:
# Sim duration
sim_duration = 3000.0

# Empty Simulation Configuration
sim_conf = obi.CircuitSimulationScanConfig.empty_config()

# Info
info = obi.Info(campaign_name="Small microcircuit simulation", campaign_description="Simulation of a small microcircuit with predefined neuron set and constant current stimulus")
sim_conf.set(info, name="info")

# Neuron Sets
sim_neuron_set = obi.IDNeuronSet(neuron_ids=obi.NamedTuple(name="IDNeuronSet1", elements=range(10)))
sim_conf.add(sim_neuron_set, name='ID10')

sync_neuron_set = obi.IDNeuronSet(neuron_ids=obi.NamedTuple(name="IDNeuronSet2", elements=range(3)))
sim_conf.add(sync_neuron_set, name='ID3')

# Regular Timesteps
regular_timestamps = obi.RegularTimestamps(start_time=0.0, number_of_repetitions=3, interval=sim_duration)
sim_conf.add(regular_timestamps, name='RegularTimestamps')

# Stimulus
poisson_input = obi.PoissonSpikeStimulus(duration=800.0, timestamps=regular_timestamps.ref, frequency=20, source_neuron_set=sim_neuron_set.ref, targeted_neuron_set=sim_neuron_set.ref)
sim_conf.add(poisson_input, name='PoissonInputStimulus')

sync_input = obi.FullySynchronousSpikeStimulus(timestamps=regular_timestamps.ref, source_neuron_set=sync_neuron_set.ref, targeted_neuron_set=sim_neuron_set.ref)
sim_conf.add(sync_input, name='SynchronousInputStimulus')

# Recordings
voltage_recording = obi.SomaVoltageRecording(neuron_set=sim_neuron_set.ref, start_time=0.0, end_time=sim_duration)
sim_conf.add(voltage_recording, name='VoltageRecording')

# Synaptic manipulations (executed in order!!)
syn_manip_mg = obi.SynapticMgManipulation(magnesium_value=[2.0, 2.4])
syn_manip_use = obi.ScaleAcetylcholineUSESynapticManipulation(use_scaling=0.7050728631217412)
sim_conf.add(syn_manip_mg, name='SynapticMgManipulation')
sim_conf.add(syn_manip_use, name='ScaleAcetylcholineUSESynapticManipulation')

# Initialization
simulations_initialize = obi.CircuitSimulationScanConfig.Initialize(circuit=circuit, 
                                                        node_set=sim_neuron_set.ref, 
                                                        simulation_length=sim_duration)
sim_conf.set(simulations_initialize, name='initialize')

# Validated Config
validated_sim_conf = sim_conf.validated_config()

In [ ]:
# sim_conf.model_dump(mode="json")

### Generate a grid scan

In [ ]:
grid_scan = obi.GridScanGenerationTask(form=validated_sim_conf, output_root='../../../../obi-output/circuit_simulations_with_manipulations/grid_scan')
grid_scan.multiple_value_parameters(display=True)
grid_scan.coordinate_parameters(display=True)
grid_scan.execute()
obi.run_tasks_for_generated_scan(grid_scan)

### Deserialize the campaign config .json

In [ ]:
# Deserialization
grid_scan_ds = obi.deserialize_obi_object_from_json_file("../../../../obi-output/circuit_simulations_with_manipulations/grid_scan/obi_one_scan.json")

In [ ]:
grid_scan_ds